In [1]:
!pip3 install transformers==4.33.2
!pip3 install optimum==1.13.2
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
Wed Jan  3 17:20:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |              

In [2]:
import pickle
import random
import itertools
import re

from tqdm.notebook import tqdm, trange
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd

In [3]:
model_name_or_path = 'TheBloke/Llama-2-13B-German-Assistant-v4-GPTQ'  # "TheBloke/Llama-2-13B-chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map='auto',
                                             trust_remote_code=False,
                                             revision="gptq-4bit-32g-actorder_True")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
with open("definitions.pickle", "rb") as file:
    definitions = pickle.load(file)
with open("sentences.pickle", "rb") as file:
    sentences = pickle.load(file)

In [5]:
def find_examples(definition: str, options: dict, temperature=0.1, top_p=0.95, top_k=1, max_new_tokens=5):
    examples = ""
    for num, option in options.items():
        examples += f"{num}. {option} "
    prompt = lambda definition, examples: f'''### User: Du bist genau und gewissenhaft und gibst stets die korrekte Antwort. Hier ist eine Definition: {definition} Hier sind nummerierte Beispiele: {examples} Nenne die Nummer des Beispiels, das zur Definition passt.
        ### Assistant: '''
    prompt_length = len(prompt(definition, examples))
    i = 100  # only try 100 times
    output = -1
    while (output < 0) and (i > 0):
        i -= 1
        input_ids = tokenizer(prompt(definition, examples), return_tensors='pt').input_ids.cuda()
        output = model.generate(inputs=input_ids, temperature=temperature,
                                do_sample=True, top_p=top_p, top_k=top_k,
                                max_new_tokens=max_new_tokens)
        output = tokenizer.decode(output[0])[prompt_length:].strip()
        output = re.findall('\d+', output)
        if len(output) > 0:
            output = int(output[0])
        print(i, end=" ")
        if i == 0:
            return -1
    print()
    return output

In [ ]:
random.seed(15)
attempts = 10

for false_positives in range(2, 6):
    result = []
    for key, definition in tqdm(definitions.items()):
        for attempt in range(attempts):
            try:
                sentence = random.choice(list(sentences[int(key)]))
            except KeyError:
                print((None, None, None))
                result.append(pd.Series({"constr": key, "definition": definition, "example": None, "prediction": None, "correct": None}))
                continue
            others = itertools.chain.from_iterable([sentence_list for constr, sentence_list in sentences.items() if int(constr) != int(key)])
            others = random.choices(list(others), k=false_positives)
            query = dict(enumerate(set(others) | {sentence}))
            prediction = find_examples(definition, query)
            print(query.get(prediction) == sentence, sentence, prediction)
            result.append(pd.Series({"constr": key, "definition": definition, "example": sentence, "prediction": query.get(prediction), "correct": query.get(prediction) == sentence}))
    result = pd.DataFrame(result)
    result.to_csv(f"result_1_vs_{false_positives}_{attempts}attempts.tsv", sep="\t")

  0%|          | 0/211 [00:00<?, ?it/s]

99 
False Solange zwischen den beiden Palästinenser-Gebilden latenter Bürgerkrieg herrscht , ist ein Zweistaaten-Model selbst in der Logik nicht auf der Agenda , geschweige denn in der Praxis . 1
99 
False Da musst du dann die juristischen Begriffe benutzen , du musst dich damit abfinden , dass dein Mandant nicht versteht , worum es geht , geschweige denn das Publikum . 1
99 
False Eine ausführliche Begründung , geschweige denn eine rechtliche Handhabe gibt es nicht . 0
99 
True Reicher , geschweige denn glücklicher , wird ein Volk dadurch nicht – ebenso wenig , wie es durch die Heirat des Geistlichen ärmer wird . 1
99 
False Dennoch hat Hamas es nicht geschafft , ihre militärischen Kapazitäten auszubauen , geschweige denn eine zivile Infrastruktur in ihrem Einflussgebiet zu etablieren . 1
99 
True Und viele Künstler kreisten derart um sich selbst , dass sich ein Blick in die Tradition der Kunst , geschweige denn über die deutsch-deutsche Grenze geradezu verbot . 0
99 
True Solange zwi